In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Command Line Tools

OpenMDAO has a number of command line tools that are available via the `openmdao`
command.

```{note}
The `openmdao` sub-commands, as well as any other console scripts associated with OpenMDAO, will
only be available if you have installed OpenMDAO using *pip*. See [Getting Started](../getting_started/getting_started.ipynb)
```

```{note}
When using a command line tool on a script that takes its own command line arguments, those
arguments must be placed after a `--` on the command line.  Anything to the right of the
`--` will be ignored by the openmdao command line parser and passed on to the user script.
For example: `openmdao n2 -o foo.html myscript.py -- -x --myarg=bar` would pass
`-x` and `--myarg=bar` as args to `myscript.py`.
```

All available `openmdao` sub-commands can be shown using the following command:

```
openmdao -h
```

In [ ]:
!openmdao -h

To get further info on any sub-command, follow the command with a *-h*.  For example:

```
openmdao n2 -h
```

In [ ]:
!openmdao n2 -h

```{note}
Several of the example commands below make use of the files `circuit.py` and `circle_opt.py`. These files are located in the openmdao/test_suite/scripts directory.
```

## Viewing and Checking Commands

Usually these commands will exit after executing, rather than continuing to the end of the user's
run script. This makes it convenient to view or check the configuration of a model in any
run script without having to wait around for the entire script to run.


### openmdao check

The `openmdao check` command will perform a number of checks on a model and display
errors, warnings, or informational messages describing what it finds. Some of the available
checks are *unconnected_inputs*, which lists any input variables that are not connected, and
*out_of_order*, which displays any systems that are being executed out-of-order.
You can supply individual checks on the command line using *-c* args.  For example:

If you are in Colab or Binder, the shell command will not find the file because it is a single notebook without the included file.

```
openmdao check -c cycles circuit.py
```

In [ ]:
!openmdao check -c cycles circuit.py

Otherwise, a set of default checks will be done. To see lists of the available and default checks, run the following command:

```
openmdao check -h
```

In [ ]:
!openmdao check -h

(om-command-n2)=
### openmdao n2

The `openmdao n2` command will generate an $N^2$ diagram of the model that is
viewable in a browser, for example:

```
openmdao n2 circuit.py
```

will generate an $N^2$ diagram like the one below.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src93", get_code("openmdao.test_suite.scripts.circuit_analysis.Circuit"), display=False)

:::{Admonition} `Circuit` class definition 
:class: dropdown

{glue:}`code_src93`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model
model.add_subsystem('circuit', Circuit())

p.setup()

om.n2(p)

```{note}
To make use of the `--problem` argument, it is helpful to give your subproblem a meaningful name when it is instantiated, which you can use to identify it on the command line. 

For example:

    subprob = om.Problem(name='subproblem1')

    subcomp = om.SubmodelComp(problem=subprob)

Then:

    openmdao n2 --problem subproblem1
```

If the test module happens to be part of a python package, then you can also use the dotted
module pathname of the test module instead of the filename.

A number of other openmdao commands, includng `view_connections` and `tree`, also support this
functionality

(om-command-view-connections)=
### openmdao view_connections

The `openmdao view_connections` command generates a table of connection information for all input and
output variables in the model. Each row represents a connection between an output and a single input. The viewer's primary purpose is to help debug a model by making the following things easier:


- Identifying unconnected inputs
- Highlighting unit conversions or missing units
- Identifying missing or unwanted implicit connections


The table can be sorted by any column by clicking on the
column header, and a column can be filtered by typing text into the 'filter column' field found
at the top of each column.  Also, any column can be shown or hidden using the toggle buttons at
the bottom of the table.  When input and output units differ, they are highlighted in
red.  In the promoted input and output columns, variables that are promoted at some level in
the model are shown in blue, while variables that are never promoted are shown in black.

Below is an example of a connection viewer for a pycycle propulsor model obtained using the command:

```
openmdao view_connections -v propulsor.py
```

![An example of a connection viewer](images/view_connections.png)

<div align="center"><b>An example of a connection viewer</b></div>
<br>

By default the promoted names columns of both inputs and outputs are shown and their absolute
names are hidden.

When showing promoted output and promoted input columns, if the promoted output name equals the
promoted input name, that means the connection is an implicit connection.  Otherwise the
connection is explicit, meaning somewhere in the model there is an explicit call to `connect`
that produced the connection.

In OpenMDAO, multiple inputs can be promoted to the same name, and by sorting the promoted inputs
column, all such inputs will be grouped together.  This can make it much easier to spot either
missing or unwanted implicit connections.

(om-command-scaling)=
### openmdao scaling
The `openmdao scaling` command generates tables of information for design variables, objectives,
and constraints, as well as a viewer that shows magnitudes of subjacobians of the total jacobian.

#### Design variable/objective/constraint tables

Any of the columns in the design variable, objective, and constraint tables can be sorted by clicking on
the header of the desired column.  Each row in a table corresponds to an individual design variable,
objective, or constraint, and if that variable happens to be an array then the row can be expanded
vertically using the "+" button on the far left to show a row for each entry in that array.  In
the constraints table, if a constraint is linear it will have a green check mark in the "linear"
column.


#### Jacobian viewer

The jacobian viewer displays magnitude information for each subjacobian of the total jacobian. It
contains one column for each design variable and one row for each objective and constraint.  If there
are linear constraints, the part of the total jacobian that depends on them will be displayed in
a separate tab.  A detailed view of a given sub-jacobian can be see by left clicking on the corresponding
cell in the total jacobian view.  It will open a new tab containing the detailed sub-jacobian view.
The detailed sub-jacobian view can be closed by right clicking on the tab.

Cells in both the top level and detailed sub-jacobian views will be colored based on the maximum
absolute value found in that location.  If the location is known to be zero because a total coloring
has been computed, it will be dark gray in color.  If the location happens to have a value of zero
for some other reason, it will be colored light gray.  All other values will be displayed using a color
map that goes from red at large values down to blue for small values.


Below is an example of what the driver scaling tables and the jacobian view look like:

![An example of driver scaling report tables](images/scaling_report_tables.png)

<div align="center"><b>An example of driver scaling report tables.</b></div>
<br>

![An example of driver scaling report jacobian view](images/scaling_report_jac.png)

<div align="center"><b>An example of driver scaling report jacobian view.</b></div>
<br>

#### openmdao tree

The `openmdao tree` command prints an indented list of all systems in the model tree.  Each system's
type and name are shown, along with their linear and nonlinear solvers if
they differ from the defaults, which are LinearRunOnce and NonlinearRunOnce respectively.
If the `-c` option is used, the tree will print in color if the terminal supports it and
the *colorama* package is installed. If colors are used, implicit and explicit components will be
displayed using different colors.

The input and output sizes can also be displayed using the `--sizes` arg, and the `--approx` arg
will display the approximation method and the number of approximated partials for systems that use
approximated derivatives.

The tree command also allows specific attributes and/or vector variables to be printed out along with their
corresponding system in the tree using the `--attr` and `--var` args respectively.

Here's an example of the tree output for a simple circuit model:

```
openmdao tree --sizes --approx circuit.py
```

In [ ]:
!openmdao tree --sizes --approx circuit.py

### openmdao summary

The `openmdao summary` command prints a high level summary of the model.  For example:

```
openmdao summary circle_opt.py
```

In [ ]:
!openmdao summary circle_opt.py

### openmdao cite

The `openmdao cite` command prints citations for any classes in the model that have them.
It supports optional `-c` arguments to allow you to limit displayed citations to
only those belonging to a particular class or group of classes.  By default, all citations for
any class used in the problem will be displayed. For example:

```
openmdao cite circuit.py
```

In [ ]:
!openmdao cite circuit.py

## Profiling and Tracing Commands

The following commands perform profiling or tracing on a run script, filtering their target
functions based on pre-defined groups of functions that can be specified using the `-g` command line option. The available group will be displayed using the `-h` command line option.  For example, here's the usage output for the `openmdao trace` command, which includes the function groups available at the time of this writing:

```
usage: openmdao trace [-h] [-g METHODS] [-v] file

positional arguments:
  file                  Python file to be traced.

optional arguments:
  -h, --help            show this help message and exit
  -g METHODS, --group METHODS
                        Determines which group of methods will be traced.
                        Default is "openmdao". Options are: ['dataflow',
                        'linear', 'mpi', 'openmdao', 'openmdao_all', 'setup']
  -v, --verbose         Show function locals and return values.

```

### openmdao iprof

The `openmdao iprof` command will display an icicle plot showing the time elapsed in all of the target
methods corresponding to each object instance that they were called on.  For more details, see
[Instance-based Profiling](../features/debugging/profiling/inst_profile.ipynb).



### openmdao iprof_totals

The `openmdao iprof_totals` command performs the same profiling as `openmdao iprof`, but it outputs a simple,
text-based summary of the total time spent in each method.  The [Instance-based Profiling](../features/debugging/profiling/inst_profile.ipynb) section contains more details.

### openmdao trace

The `openmdao trace` command prints a call trace for a specified set of functions.  Optionally it can
display values of function locals and return values.  For more detail, see [Instance-based Call Tracing](../features/debugging/profiling/inst_call_tracing.ipynb).


### openmdao timing

The `openmdao timing` command, like the `openmdao iprof` command, is meant to show how much time is being spent in methods of particular instances under MPI.  The `timing` command focuses specifically on methods of openmdao components and groups, and it also has features to make it easier to understand how an openmdao model is performing when running under MPI.  Aside from the addition of MPI related information, it has the added advantage over `openmdao iprof` that its overhead is much lower so it shouldn't increase run time very much, and it shouldn't distort the timing results as can happen with `openmdao iprof`. For more details, see
[Timing Systems under MPI](../features/debugging/profiling/timing.ipynb).


Memory Profiling

### openmdao mem

The `openmdao mem` command profiles the memory usage of python functions.  For more detail,
see [Memory Profiling](../features/debugging/profiling/inst_mem_profile.ipynb).

### openmdao mempost

The `openmdao mempost` command postprocesses the raw memory dump file generated by `openmdao mem`.
For more detail, see [Memory Profiling](../features/debugging/profiling/inst_mem_profile.ipynb).



## Other Commands

### openmdao call_tree

The `openmdao call_tree` command takes the full module path of a class method and displays the
call tree for that method.  It's purpose is to show which class 'owns' the specified method
call and any other 'self.*' methods that it calls. Note that it shows all of the methods called,
regardless of the result of conditionals within any function, so the displayed tree does not
necessarily represent a trace of the function as it executes.  The functions are ordered top to
bottom as they are encountered in the source code, and a given subfunction is only displayed
once within a given function, even if it is actually called in multiple places within the function.
Here's an example:

```
openmdao call_tree openmdao.api.LinearBlockGS.solve
```

In [ ]:
!openmdao call_tree openmdao.api.LinearBlockGS.solve

(ref-scaffold)=
### openmdao scaffold

The `openmdao scaffold` command generates simple scaffolding, or 'skeleton' code for
a class that inherits from an allowed OpenMDAO base class.  The allowed base classes are shown as
part of the description of the `--base` arg below:

```
openmdao scaffold -h
```

In [ ]:
!openmdao scaffold -h

In addition, the command will generate the scaffolding for a simple
test file for that class, and if the `--package` option is used, it will generate the directory
structure for a simple installable python package and will declare an entry point in the
`setup.py` file so that the given class can be discoverable as an OpenMDAO plugin when installed.

To build scaffolding for an OpenMDAO command line tool plugin, use the `--cmd` option.

(list-installed)=
### openmdao list_installed

The `openmdao list_installed` command lists installed classes of the specified type(s).
Its options are shown below:

```
openmdao list_installed -h
```

In [ ]:
!openmdao list_installed -h

By default, installed types from all installed packages are shown, but the output can be filtered
by the use of the `-i` option to include only specified packages, or the `-x` option
to exclude specified packages.

For example, to show only those linear and nonlinear solver types that are part of the `openmdao`
package, do the following:

```
openmdao list_installed lin_solver nl_solver -i openmdao
```

In [ ]:
!openmdao list_installed lin_solver nl_solver -i openmdao

Similarly, to hide all of the built-in (openmdao) solver types and only see installed plugin
solver types, do the following.

```
openmdao list_installed lin_solver nl_solver -x openmdao
```

### openmdao find_repos

The `openmdao find_repos` command finds github repositories with `openmdao` or openmdao plugin specific
strings registered as topics.

Its options are shown below:

```
openmdao find_repos -h
```

In [ ]:
!openmdao find_repos -h

One example of its use would be to display any github repositories claiming to contain openmdao command
line tools.  At the time this documentation was created, the following repositories were found:

```
openmdao find_repos command
```

In [ ]:
!openmdao find_repos command

(compute-entry-points)=
### openmdao compute_entry_points

The `openmdao compute_entry_points` command lists entry point groups and entry points for
any openmdao compatible classes, e.g., Component, Group, etc., that it finds within a given
python package. Its options are shown below:

```
openmdao compute_entry_points -h
```

In [ ]:
!openmdao compute_entry_points -h

For example, to show all of the potential openmdao entry point groups and entry points for an
installed python package called `mypackage`, you would do the following:

```
openmdao compute_entry_points mypackage
```

The entry point information will be printed in a form that can easily be pasted into the
`setup.py` file for the specified package.

## Using Commands under MPI

In general, usage of openmdao subcommands under MPI is the same as usual, except the command will
be preceded by `mpirun -n <num_procs>`.  For example:

```
mpirun -n 2 openmdao summary multipoint_beam_opt.py
```

In [ ]:
!mpirun -n 2 openmdao summary multipoint_beam_opt.py

```
============== Problem Summary ============
Groups:               4
Components:          10
Max tree depth:       3

Design variables:            1   Total size:        5

Nonlinear Constraints:       1   Total size:        1
    equality:                1                      1
    inequality:              0                      0

Linear Constraints:          0   Total size:        0
    equality:                0                      0
    inequality:              0                      0

Objectives:                  1   Total size:        1

Input variables:            10   Total size:     1961
Output variables:           10   Total size:     1117

Total connections: 10   Total transfer data size: 1961

Driver type: ScipyOptimizeDriver
Linear Solvers: [LinearRunOnce x 4]
Nonlinear Solvers: [NonlinearRunOnce x 4]
```